In [3]:
import math
from datetime import datetime,timedelta
import numpy as np
import pandas
import random
import preprocess_functions

In [4]:
df1 = pandas.read_csv("training_20min_avg_volume_etc_cartype.csv")
df2 = pandas.read_csv("training2_20min_avg_volume_etc_cartype.csv")
df = df1.append(df2, ignore_index=True)
df_time_temp = df['time_window'].str.split('[',expand=True)[1].str.split(',',expand=True)[0]
df['time_start'] = df_time_temp

In [6]:
df_1_0 = preprocess_functions.extract_tg_dir(df, 1, 0)
df_1_1 = preprocess_functions.extract_tg_dir(df, 1, 1)
df_2_0 = preprocess_functions.extract_tg_dir(df, 2, 0)
df_3_0 = preprocess_functions.extract_tg_dir(df, 3, 0)
df_3_1 = preprocess_functions.extract_tg_dir(df, 3, 1)

In [7]:
df_filled = pandas.read_csv('./dataSet/phase2_filled_vacancy.csv')
df_1_0_update = preprocess_functions.update_vacancy_volume(df_1_0, df_filled)
df_1_1_update = preprocess_functions.update_vacancy_volume(df_1_1, df_filled)
df_2_0_update = preprocess_functions.update_vacancy_volume(df_2_0, df_filled)
df_3_0_update = preprocess_functions.update_vacancy_volume(df_3_0, df_filled)
df_3_1_update = preprocess_functions.update_vacancy_volume(df_3_1, df_filled)

In [8]:
df_1_0_new = preprocess_functions.add_last2h_dimension(df_1_0_update, 'volume')
df_1_1_new = preprocess_functions.add_last2h_dimension(df_1_1_update, 'volume')
df_2_0_new = preprocess_functions.add_last2h_dimension(df_2_0_update, 'volume')
df_3_0_new = preprocess_functions.add_last2h_dimension(df_3_0_update, 'volume')
df_3_1_new = preprocess_functions.add_last2h_dimension(df_3_1_update, 'volume')
for i in xrange(8):
    df_1_0_new = preprocess_functions.add_last2h_dimension(df_1_0_new, 'model'+'%d'%i)
    df_1_1_new = preprocess_functions.add_last2h_dimension(df_1_1_new, 'model'+'%d'%i)
    df_2_0_new = preprocess_functions.add_last2h_dimension(df_2_0_new, 'model'+'%d'%i)
    df_3_0_new = preprocess_functions.add_last2h_dimension(df_3_0_new, 'model'+'%d'%i)
    df_3_1_new = preprocess_functions.add_last2h_dimension(df_3_1_new, 'model'+'%d'%i)
for i in xrange(2):
    df_1_0_new = preprocess_functions.add_last2h_dimension(df_1_0_new, 'type'+'%d'%i)
    df_1_1_new = preprocess_functions.add_last2h_dimension(df_1_1_new, 'type'+'%d'%i)
    df_2_0_new = preprocess_functions.add_last2h_dimension(df_2_0_new, 'type'+'%d'%i)
    df_3_0_new = preprocess_functions.add_last2h_dimension(df_3_0_new, 'type'+'%d'%i)
    df_3_1_new = preprocess_functions.add_last2h_dimension(df_3_1_new, 'type'+'%d'%i)
df_1_0_new = preprocess_functions.add_last2h_dimension(df_1_0_new, 'has_etc1')
df_1_1_new = preprocess_functions.add_last2h_dimension(df_1_1_new, 'has_etc1')
df_2_0_new = preprocess_functions.add_last2h_dimension(df_2_0_new, 'has_etc1')
df_3_0_new = preprocess_functions.add_last2h_dimension(df_3_0_new, 'has_etc1')
df_3_1_new = preprocess_functions.add_last2h_dimension(df_3_1_new, 'has_etc1')

In [9]:
frame = [df_1_0_new,df_1_1_new,df_2_0_new,df_3_0_new,df_3_1_new]
df_full = pandas.concat(frame,ignore_index=True).drop(['time_window'],axis=1)

In [10]:
df_full_temp = df_full
length_full = len(df_full['time_start'])
length_filled = len(df_filled['time_start'])
for i in xrange(length_full):
    temp_full = df_full_temp['time_start'][i]
    tg_full = df_full_temp['tollgate_id'][i]
    dir_full = df_full_temp['direction'][i]
    for j in xrange(length_filled):
        if (temp_full == df_filled['time_start'][j]) and (tg_full == df_filled['tollgate_id'][j])\
                                                     and (dir_full == df_filled['direction'][j]):
            df_full_temp = df_full_temp.drop(i)
df_full_temp = df_full_temp.reset_index().drop('index',axis=1)

In [11]:
df_weather1 = pandas.read_csv("./dataSet/v3_weather (table 7)_training.csv")
df_weather2 = pandas.read_csv("./dataSet/v2_weather (table 7)_test.csv")
df_weather_temp = df_weather1.append(df_weather2, ignore_index=True)

In [36]:
df_time_temp = df_full_temp['time_start']
df_insert_weather = preprocess_functions.insert_weather(df_full_temp, df_time_temp, df_weather_temp)
df_add_time = preprocess_functions.add_time_dimension(df_insert_weather, df_time_temp)
df_h_judge_raw = preprocess_functions.holiday_judge(df_add_time, df_time_temp)
df_h_judge = preprocess_functions.holiday_judge_out(df_h_judge_raw, df_time_temp)
df_h_judge_new = preprocess_functions.holiday_judge_in(df_h_judge, df_time_temp)
df_final = preprocess_functions.transfer_volume(df_h_judge_new)

In [37]:
log_base = np.e

In [38]:
df_final['volumelv0_log'] = np.log(df_final['volumelv0'])/np.log(log_base)
df_final['volumelv1_log'] = np.log(df_final['volumelv1'])/np.log(log_base)
df_final['volumelv2_log'] = np.log(df_final['volumelv2'])/np.log(log_base)
df_final['volumelv3_log'] = np.log(df_final['volumelv3'])/np.log(log_base)
df_final['volumelv4_log'] = np.log(df_final['volumelv4'])/np.log(log_base)
df_final['volumelv5_log'] = np.log(df_final['volumelv5'])/np.log(log_base)
df_final['has_etc1lv0_log'] = np.log(df_final['has_etc1lv0']+1)/np.log(log_base)
df_final['has_etc1lv1_log'] = np.log(df_final['has_etc1lv1']+1)/np.log(log_base)
df_final['has_etc1lv2_log'] = np.log(df_final['has_etc1lv2']+1)/np.log(log_base)
df_final['has_etc1lv3_log'] = np.log(df_final['has_etc1lv3']+1)/np.log(log_base)
df_final['has_etc1lv4_log'] = np.log(df_final['has_etc1lv4']+1)/np.log(log_base)
df_final['has_etc1lv5_log'] = np.log(df_final['has_etc1lv5']+1)/np.log(log_base)
df_final['type0lv0_log'] = np.log(df_final['type0lv0']+1)/np.log(log_base)
df_final['type0lv1_log'] = np.log(df_final['type0lv1']+1)/np.log(log_base)
df_final['type0lv2_log'] = np.log(df_final['type0lv2']+1)/np.log(log_base)
df_final['type0lv3_log'] = np.log(df_final['type0lv3']+1)/np.log(log_base)
df_final['type0lv4_log'] = np.log(df_final['type0lv4']+1)/np.log(log_base)
df_final['type0lv5_log'] = np.log(df_final['type0lv5']+1)/np.log(log_base)
df_final['model1lv0_log'] = np.log(df_final['model1lv0']+1)/np.log(log_base)
df_final['model1lv1_log'] = np.log(df_final['model1lv1']+1)/np.log(log_base)
df_final['model1lv2_log'] = np.log(df_final['model1lv2']+1)/np.log(log_base)
df_final['model1lv3_log'] = np.log(df_final['model1lv3']+1)/np.log(log_base)
df_final['model1lv4_log'] = np.log(df_final['model1lv4']+1)/np.log(log_base)
df_final['model1lv5_log'] = np.log(df_final['model1lv5']+1)/np.log(log_base)

In [39]:
preprocess_functions.insert_onehot(df_final, 'tollgate_id')
preprocess_functions.insert_onehot(df_final, 'month')
preprocess_functions.insert_onehot(df_final, 'weekday')
preprocess_functions.insert_onehot(df_final, 'time')
preprocess_functions.insert_onehot(df_final, 'precipitation')

,precipitation2,precipitation1,precipitation0,time71,time70,time69,time68,time67,time66,time65,...,type0lv2_log,type0lv3_log,type0lv4_log,type0lv5_log,model1lv0_log,model1lv1_log,model1lv2_log,model1lv3_log,model1lv4_log,model1lv5_log
0,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
1,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
2,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
3,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
4,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
5,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
6,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
7,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
8,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
9,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585


In [40]:
df_final.to_csv('phase2_trainingset.csv', index=False)

# 10 KNN(need to replace df_full_temp with df_full first, or the length of element is wrong)

In [41]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

In [42]:
df_test_last2h = pandas.read_csv('test2_20min_avg_volume_etc_cartype.csv')
df_test_last2h
df_last2h_temp = df_test_last2h['time_window'].str.split('[',expand=True)[1].str.split(',',expand=True)[0]
df_test_last2h['time_start'] = df_last2h_temp
df_last2h_time = preprocess_functions.add_time_dimension(df_test_last2h, df_test_last2h['time_start'])
df_last2h_time['date_temp'] = df_last2h_time['time_start'].str.split('-',expand=True)[2]\
                               .str.split(' ',expand=True)[0].apply(int)

In [43]:
df_last2h_time_10 = df_last2h_time[(df_last2h_time['tollgate_id']==1) & (df_last2h_time['direction']==0)]
df_last2h_time_11 = df_last2h_time[(df_last2h_time['tollgate_id']==1) & (df_last2h_time['direction']==1)]
df_last2h_time_20 = df_last2h_time[(df_last2h_time['tollgate_id']==2) & (df_last2h_time['direction']==0)]
df_last2h_time_30 = df_last2h_time[(df_last2h_time['tollgate_id']==3) & (df_last2h_time['direction']==0)]
df_last2h_time_31 = df_last2h_time[(df_last2h_time['tollgate_id']==3) & (df_last2h_time['direction']==1)]

In [44]:
predict_list_10m = []
predict_list_11m = []
predict_list_20m = []
predict_list_30m = []
predict_list_31m = []

for i in xrange(7):
    df_last2h_date_10m = df_last2h_time_10[(df_last2h_time_10['date_temp'] == (25+i)) & (df_last2h_time_10['time']<24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_10m.append(list(df_last2h_date_10m['volume']))
for i in xrange(7):
    df_last2h_date_11m = df_last2h_time_11[(df_last2h_time_11['date_temp'] == (25+i)) & (df_last2h_time_11['time']<24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_11m.append(list(df_last2h_date_11m['volume']))
for i in xrange(7):
    df_last2h_date_20m = df_last2h_time_20[(df_last2h_time_20['date_temp'] == (25+i)) & (df_last2h_time_20['time']<24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_20m.append(list(df_last2h_date_20m['volume']))
for i in xrange(7):
    df_last2h_date_30m = df_last2h_time_30[(df_last2h_time_30['date_temp'] == (25+i)) & (df_last2h_time_30['time']<24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_30m.append(list(df_last2h_date_30m['volume']))
for i in xrange(7):
    df_last2h_date_31m = df_last2h_time_31[(df_last2h_time_31['date_temp'] == (25+i)) & (df_last2h_time_31['time']<24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_31m.append(list(df_last2h_date_31m['volume']))
predict_list_10m

[[10, 17, 30, 25, 38, 44],
 [12, 9, 25, 31, 32, 45],
 [14, 14, 23, 20, 36, 42],
 [8, 21, 14, 28, 37, 46],
 [11, 17, 19, 27, 37, 24],
 [18, 25, 30, 49, 40, 74],
 [27, 54, 60, 95, 98, 106]]

In [45]:
predict_list_10a = []
predict_list_11a = []
predict_list_20a = []
predict_list_30a = []
predict_list_31a = []

for i in xrange(7):
    df_last2h_date_10a = df_last2h_time_10[(df_last2h_time_10['date_temp'] == (25+i)) & (df_last2h_time_10['time']>24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_10a.append(list(df_last2h_date_10a['volume']))
for i in xrange(7):
    df_last2h_date_11a = df_last2h_time_11[(df_last2h_time_11['date_temp'] == (25+i)) & (df_last2h_time_11['time']>24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_11a.append(list(df_last2h_date_11a['volume']))
for i in xrange(7):
    df_last2h_date_20a = df_last2h_time_20[(df_last2h_time_20['date_temp'] == (25+i)) & (df_last2h_time_20['time']>24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_20a.append(list(df_last2h_date_20a['volume']))
for i in xrange(7):
    df_last2h_date_30a = df_last2h_time_30[(df_last2h_time_30['date_temp'] == (25+i)) & (df_last2h_time_30['time']>24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_30a.append(list(df_last2h_date_30a['volume']))
for i in xrange(7):
    df_last2h_date_31a = df_last2h_time_31[(df_last2h_time_31['date_temp'] == (25+i)) & (df_last2h_time_31['time']>24)]\
                                .sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
    predict_list_31a.append(list(df_last2h_date_31a['volume']))
predict_list_10a

[[52, 44, 46, 50, 40, 50],
 [38, 36, 44, 49, 45, 46],
 [40, 37, 45, 49, 41, 45],
 [62, 48, 57, 78, 67, 58],
 [60, 48, 52, 48, 62, 48],
 [122, 129, 152, 118, 112, 110],
 [94, 94, 88, 85, 93, 88]]

In [52]:
df_final['date_temp'] = df_final['time_start'].str.split('-',expand=True)[2]\
                               .str.split(' ',expand=True)[0].apply(int)
d10 = df_final[(df_final['tollgate_id']==1) & (df_final['direction']==0)]
d11 = df_final[(df_final['tollgate_id']==1) & (df_final['direction']==1)]
d20 = df_final[(df_final['tollgate_id']==2) & (df_final['direction']==0)]
d30 = df_final[(df_final['tollgate_id']==3) & (df_final['direction']==0)]
d31 = df_final[(df_final['tollgate_id']==3) & (df_final['direction']==1)]

knn_training_list_10m = []
knn_training_list_11m = []
knn_training_list_20m = []
knn_training_list_30m = []
knn_training_list_31m = []

knn_training_list_10a = []
knn_training_list_11a = []
knn_training_list_20a = []
knn_training_list_30a = []
knn_training_list_31a = []

#morning
for i in xrange(12):
    df_last2h_date_10m = d10[(d10['date_temp'] == (19+i)) & (d10['month'] == 9) & (d10['time'] > 17) &\
                               (d10['time'] < 24) ]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_10m.append(list(df_last2h_date_10m['volume']))
for i in xrange(17):
    df_last2h_date_10m = d10[(d10['date_temp'] == (8+i)) & (d10['month'] == 10) & (d10['time'] > 17) &\
                               (d10['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_10m.append(list(df_last2h_date_10m['volume']))
    
for i in xrange(12):
    df_last2h_date_11m = d11[(d11['date_temp'] == (19+i)) & (d11['month'] == 9) & (d11['time'] > 17) &\
                               (d11['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_11m.append(list(df_last2h_date_11m['volume']))
for i in xrange(17):
    df_last2h_date_11m = d11[(d11['date_temp'] == (8+i)) & (d11['month'] == 10) & (d11['time'] > 17) &\
                               (d11['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_11m.append(list(df_last2h_date_11m['volume']))
    
for i in xrange(12):
    df_last2h_date_20m = d20[(d20['date_temp'] == (19+i)) & (d20['month'] == 9) & (d20['time'] > 17) &\
                               (d20['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_20m.append(list(df_last2h_date_20m['volume']))
for i in xrange(17):
    df_last2h_date_20m = d20[(d20['date_temp'] == (8+i)) & (d20['month'] == 10) & (d20['time'] > 17) &\
                               (d20['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_20m.append(list(df_last2h_date_20m['volume']))
    
for i in xrange(12):
    df_last2h_date_30m = d30[(d30['date_temp'] == (19+i)) & (d30['month'] == 9) & (d30['time'] > 17) &\
                               (d30['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_30m.append(list(df_last2h_date_30m['volume']))
for i in xrange(17):
    df_last2h_date_30m = d30[(d30['date_temp'] == (8+i)) & (d30['month'] == 10) & (d30['time'] > 17) &\
                               (d30['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_30m.append(list(df_last2h_date_30m['volume'])) 

for i in xrange(12):
    df_last2h_date_31m = d31[(d31['date_temp'] == (19+i)) & (d31['month'] == 9) & (d31['time'] > 17) &\
                               (d31['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_31m.append(list(df_last2h_date_31m['volume']))
for i in xrange(17):
    df_last2h_date_31m = d31[(d31['date_temp'] == (8+i)) & (d31['month'] == 10) & (d31['time'] > 17) &\
                               (d31['time'] < 24)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_31m.append(list(df_last2h_date_31m['volume']))
    
#afternoon
for i in xrange(12):
    df_last2h_date_10a = d10[(d10['date_temp'] == (19+i)) & (d10['month'] == 9) 
                               & (d10['time'] > 44) & (d10['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_10a.append(list(df_last2h_date_10a['volume']))
for i in xrange(17):
    df_last2h_date_10a = d10[(d10['date_temp'] == (8+i)) & (d10['month'] == 10) 
                               & (d10['time'] > 44) & (d10['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_10a.append(list(df_last2h_date_10a['volume']))
    
for i in xrange(12):
    df_last2h_date_11a = d11[(d11['date_temp'] == (19+i)) & (d11['month'] == 9)
                               & (d11['time'] > 44) & (d11['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_11a.append(list(df_last2h_date_11a['volume']))
for i in xrange(17):
    df_last2h_date_11a = d11[(d11['date_temp'] == (8+i)) & (d11['month'] == 10) 
                               & (d11['time'] > 44) & (d11['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_11a.append(list(df_last2h_date_11a['volume']))
    
for i in xrange(12):
    df_last2h_date_20a = d20[(d20['date_temp'] == (19+i)) & (d20['month'] == 9) 
                               & (d20['time'] > 44) & (d20['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_20a.append(list(df_last2h_date_20a['volume']))
for i in xrange(17):
    df_last2h_date_20a = d20[(d20['date_temp'] == (8+i)) & (d20['month'] == 10) 
                                & (d20['time'] > 44) & (d20['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_20a.append(list(df_last2h_date_20a['volume']))
    
for i in xrange(12):
    df_last2h_date_30a = d30[(d30['date_temp'] == (19+i)) & (d30['month'] == 9)
                                & (d30['time'] > 44) & (d30['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_30a.append(list(df_last2h_date_30a['volume']))
for i in xrange(17):
    df_last2h_date_30a = d30[(d30['date_temp'] == (8+i)) & (d30['month'] == 10)
                               & (d30['time'] > 44) & (d30['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_30a.append(list(df_last2h_date_30a['volume'])) 

for i in xrange(12):
    df_last2h_date_31a = d31[(d31['date_temp'] == (19+i)) & (d31['month'] == 9)
                                & (d31['time'] > 44) & (d31['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_31a.append(list(df_last2h_date_31a['volume']))
for i in xrange(17):
    df_last2h_date_31a = d31[(d31['date_temp'] == (8+i)) & (d31['month'] == 10)
                                & (d31['time'] > 44) & (d31['time'] < 51)]\
                               .sort_values(['time','tollgate_id','direction'])\
                               .reset_index().drop(['index'],axis=1)
    knn_training_list_31a.append(list(df_last2h_date_31a['volume']))
training_label = [i for i in xrange(29)]
knn_training_list_20m
#training_label

[[30.0, 36.0, 37.0, 84.0, 80.0, 97.0],
 [33.0, 31.0, 54.0, 79.0, 86.0, 93.0],
 [6.0, 9.0, 14.0, 42.0, 37.0, 44.0],
 [30.0, 31.0, 38.0, 72.0, 79.0, 100.0],
 [31.0, 32.0, 54.0, 82.0, 91.0, 103.0],
 [30.0, 31.0, 45.0, 46.0, 64.0, 82.0],
 [22.0, 23.0, 29.0, 30.0, 45.0, 62.0],
 [25.0, 40.0, 73.0, 88.0, 114.0, 126.0],
 [28.0, 31.0, 43.0, 51.0, 89.0, 104.0],
 [5.0, 13.0, 26.0, 34.0, 48.0, 34.0],
 [15.0, 14.0, 32.0, 68.0, 72.0, 88.0],
 [35.0, 37.0, 52.0, 77.0, 86.0, 104.0],
 [25.0, 27.0, 37.0, 72.0, 77.0, 97.0],
 [29.0, 31.0, 45.0, 61.0, 99.0, 100.0],
 [23.0, 34.0, 49.0, 82.0, 114.0, 115.0],
 [25.0, 31.0, 52.0, 94.0, 109.0, 103.0],
 [32.0, 27.0, 45.0, 91.0, 92.0, 119.0],
 [23.0, 54.0, 66.0, 91.0, 107.0, 145.0],
 [20.0, 45.0, 47.0, 87.0, 96.0, 108.0],
 [21.0, 13.0, 50.0, 81.0],
 [12.0, 34.0, 34.0, 40.0, 69.0, 58.0],
 [31.0, 45.0, 52.0, 84.0, 125.0, 119.0],
 [24.0, 36.0, 46.0, 83.0, 87.0, 97.0],
 [26.0, 45.0, 47.0, 87.0, 95.0, 99.0],
 [21.0, 23.0, 42.0, 76.0, 89.0, 112.0],
 [16.0, 24.0, 39.0, 66

In [47]:
neigh10m = KNeighborsClassifier(n_neighbors=1)
neigh10m.fit(knn_training_list_10m, training_label)
neigh10m.predict(predict_list_10m)

array([ 4, 11, 23, 15, 12, 21,  9])

In [48]:
neigh11m = KNeighborsClassifier(n_neighbors=1)
neigh11m.fit(knn_training_list_11m, training_label) 
neigh11m.predict(predict_list_11m)

array([14, 22, 15,  3, 13,  6, 17])

In [49]:
neigh20m = KNeighborsClassifier(n_neighbors=1)
neigh20m.fit(knn_training_list_20m, training_label)
neigh20m.predict(predict_list_20m)

C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: setting an array element with a sequence.

In [27]:
neigh30m = KNeighborsClassifier(n_neighbors=1)
neigh30m.fit(knn_training_list_30m, training_label) 
neigh30m.predict(predict_list_30m)

array([ 9, 12, 24, 12,  6, 20, 21])

In [28]:
neigh31m = KNeighborsClassifier(n_neighbors=1)
neigh31m.fit(knn_training_list_31m, training_label) 
neigh31m.predict(predict_list_31m)

array([ 3, 25, 22, 13, 10, 10,  4])

In [29]:
neigh10a = KNeighborsClassifier(n_neighbors=1)
neigh10a.fit(knn_training_list_10a, training_label) 
neigh10a.predict(predict_list_10a)

array([ 2, 12, 24,  6,  3, 11,  9])

In [30]:
neigh11a = KNeighborsClassifier(n_neighbors=1)
neigh11a.fit(knn_training_list_11a, training_label) 
neigh11a.predict(predict_list_11a)

array([16, 21,  2, 20, 21, 27, 21])

In [31]:
neigh20a = KNeighborsClassifier(n_neighbors=1)
neigh20a.fit(knn_training_list_20a, training_label) 
neigh20a.predict(predict_list_20a)

array([25,  0, 13,  4, 10,  9,  9])

In [32]:
neigh30a = KNeighborsClassifier(n_neighbors=1)
neigh30a.fit(knn_training_list_30a, training_label) 
neigh30a.predict(predict_list_30a)

array([ 1, 20, 22,  4, 21, 18, 23])

In [33]:
neigh31a = KNeighborsClassifier(n_neighbors=1)
neigh31a.fit(knn_training_list_31a, training_label) 
neigh31a.predict(predict_list_31a)

array([ 0,  0, 13,  6, 15, 20, 22])